In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [ ]:
%%time
# loading a sample from the dataset specified in config, in this case 'HuggingFaceFW/fineweb'
from tools import get_data_loaders
train_data_loader, val_data_loader = get_data_loaders(
    'HuggingFaceFW/fineweb-edu', 
    batch_size=1, 
    streaming=True,
    subset_name = "CC-MAIN-2024-10" # using a different subset of fineweb from what we'll be training the model on
)

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
%%time
batch = next(train_data_loader)
print(len(batch), batch[0])

In [4]:
# turn it into one string instead of a list of strings
combined_string = '\n\n'.join(batch)

# Convert the string to bytes
combined_bytes = combined_string.encode('utf-8')
print(len(combined_bytes))

print(combined_string[:100])
print([b for b in combined_bytes[:100]])
print(combined_bytes.decode('utf-8')[:100])

446616
For those unfamiliar with Cornish, it is classed as a p-Celtic member of the family of Celtic langua
[70, 111, 114, 32, 116, 104, 111, 115, 101, 32, 117, 110, 102, 97, 109, 105, 108, 105, 97, 114, 32, 119, 105, 116, 104, 32, 67, 111, 114, 110, 105, 115, 104, 44, 32, 105, 116, 32, 105, 115, 32, 99, 108, 97, 115, 115, 101, 100, 32, 97, 115, 32, 97, 32, 112, 45, 67, 101, 108, 116, 105, 99, 32, 109, 101, 109, 98, 101, 114, 32, 111, 102, 32, 116, 104, 101, 32, 102, 97, 109, 105, 108, 121, 32, 111, 102, 32, 67, 101, 108, 116, 105, 99, 32, 108, 97, 110, 103, 117, 97]
For those unfamiliar with Cornish, it is classed as a p-Celtic member of the family of Celtic langua


# Regex
this is a pre-processing stage where we set the rules for what types of characters are allowed to be merged together

In [5]:
import regex as re

In [6]:
# don't ask me the specifics of how this plays out, i just know it's what they used for GPT4
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# if you want to mess around with building your own tokenizer, then ^this string is one of the things to mess around with

In [7]:
compiled_pattern = re.compile(GPT4_SPLIT_PATTERN, re.UNICODE)
print(compiled_pattern)

regex.Regex("'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*|\\s*[\\r\\n]|\\s+(?!\\S)|\\s+", flags=regex.V0)


In [8]:
# split the text up into text chunks
text_chunks = re.findall(compiled_pattern, combined_string)
print(len(combined_string), len(text_chunks))

1056254 207679


In [9]:
print(combined_string[:100])
print(text_chunks[:100])

The Independent Jane
For all the love, romance and scandal in Jane Austen’s books, what they are rea
['The', ' Independent', ' Jane', '\n', 'For', ' all', ' the', ' love', ',', ' romance', ' and', ' scandal', ' in', ' Jane', ' Austen', '’s', ' books', ',', ' what', ' they', ' are', ' really', ' about', ' is', ' freedom', ' and', ' independence', '.', ' Independence', ' of', ' thought', ' and', ' the', ' freedom', ' to', ' choose', '.\n', 'Elizabeth', '’s', ' refusal', ' of', ' Mr', '.', ' Collins', ' offer', ' of', ' marriage', ' showed', ' an', ' independence', ' seldom', ' seen', ' in', ' heroines', ' of', ' the', ' day', '.', ' Her', ' refusal', ' of', ' Mr', '.', ' Darcy', ' while', ' triggered', ' by', ' anger', ' showed', ' a', ' level', ' of', ' independence', ' that', ' left', ' him', ' shocked', ' and', ' stunned', '.\n', 'The', ' freedom', ' she', ' exhibited', ' in', ' finally', ' accepting', ' him', ' in', ' direct', ' defiance', ' of', ' Lady', ' Catherine', ' and', ' know

In [10]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks] 
ids_backup = ids # saving this for later just to see how much compression we get
print(len(ids), ids[:100])

207679 [[84, 104, 101], [32, 73, 110, 100, 101, 112, 101, 110, 100, 101, 110, 116], [32, 74, 97, 110, 101], [10], [70, 111, 114], [32, 97, 108, 108], [32, 116, 104, 101], [32, 108, 111, 118, 101], [44], [32, 114, 111, 109, 97, 110, 99, 101], [32, 97, 110, 100], [32, 115, 99, 97, 110, 100, 97, 108], [32, 105, 110], [32, 74, 97, 110, 101], [32, 65, 117, 115, 116, 101, 110], [226, 128, 153, 115], [32, 98, 111, 111, 107, 115], [44], [32, 119, 104, 97, 116], [32, 116, 104, 101, 121], [32, 97, 114, 101], [32, 114, 101, 97, 108, 108, 121], [32, 97, 98, 111, 117, 116], [32, 105, 115], [32, 102, 114, 101, 101, 100, 111, 109], [32, 97, 110, 100], [32, 105, 110, 100, 101, 112, 101, 110, 100, 101, 110, 99, 101], [46], [32, 73, 110, 100, 101, 112, 101, 110, 100, 101, 110, 99, 101], [32, 111, 102], [32, 116, 104, 111, 117, 103, 104, 116], [32, 97, 110, 100], [32, 116, 104, 101], [32, 102, 114, 101, 101, 100, 111, 109], [32, 116, 111], [32, 99, 104, 111, 111, 115, 101], [46, 10], [69, 108, 105, 122, 

so this regex just splits the text up into all the token ids that are allowed to be merged, meaning that the regex output we saw above is an upper limit on the tokens that we could end up with if we get a large enough vocabulary, rather than a starting point. 

# BPE tokenization

In [11]:
vocab_size = 1024 - 3 # the desired final vocabulary size. -3 for the three special tokens bos, eos, & pad
num_merges = vocab_size - 256

In [12]:
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [13]:
# now let's actually do it
merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    #print(f"merge {i+1}/{num_merges}: {pair} -> {idx} had {stats[pair]} occurrences")

100%|█████████████████████████████████| 765/765 [03:15<00:00,  3.92it/s]


In [14]:
og = sum([len(t) for t in ids_backup])
new = sum([len(t) for t in (ids)])
print("original length:", og)
print("ids length:", new)
print(f"compression ratio: {og / new:.2f}X")

original length: 1060598
ids length: 417949
compression ratio: 2.54X


In [15]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(ids[0], decode(ids[0]))

[456] The


In [16]:
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("Once upon a time,"))

[79, 110, 317, 596, 265, 257, 662, 44]


In [19]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {'merges': merges}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [18]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and then one above it multiple times until i got to the smallest possible size (512 - 3 = 509)
vocab_size = ((vocab_size + 3) // 2) - 3 # the -3's account for our special tokens
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [1]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 1024
tokenizer_data = load_tokenizer_data(f'models/{vocab_size-3}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['merges'])

In [2]:
prompt = 'The best and most reliable form of research is the double-blind, placebo-controlled study.'
print(tokenizer.display(prompt))

tokens = tokenizer.encode(prompt)
print(tokens)

decoded = tokenizer.decode(tokens)
print(decoded)

['The', ' b', 'est', ' and', ' most', ' rel', 'i', 'able', ' form', ' of', ' rese', 'arch', ' is', ' the', ' d', 'ou', 'b', 'le', '-', 'b', 'l', 'ind', ',', ' pl', 'ace', 'b', 'o', '-', 'con', 't', 'rol', 'led', ' stud', 'y', '.']
[1021, 456, 278, 402, 285, 746, 808, 105, 550, 848, 277, 969, 953, 311, 262, 288, 286, 98, 292, 45, 98, 108, 608, 44, 458, 613, 98, 111, 45, 659, 116, 869, 723, 602, 121, 46]
The best and most reliable form of research is the double-blind, placebo-controlled study.


In [3]:
for i in range(vocab_size-3):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: ' '
1: ''
2: ''
3: ''
4: ''
5: ''
6: ''
7: ''
8: '
9: '	'
10: '
'
11: ''
12: ''
'3: '
14: ''
15: ''
16: ''
17: ''
18: ''
19: ''
20: ''
21: ''
22: ''
23: ''
24: ''
25: ''
26: ''
27: ''
28: ''
29: ''
30: ''
31: ''
32: ' '
33: '!'
34: '"'
35: '#'
36: '$'
37: '%'
38: '&'
39: '''
40: '('
41: ')'
42: '*'
43: '+'
44: ','
45: '-'
46: '.'
47: '/'
48: '0'
49: '1'
50: '2'
51: '3'
52: '4'
53: '5'
54: '6'
55: '7'
56: '8'
57: '9'
58: ':'
59: ';'
60: '<'
61: '='
62: '>'
63: '?'
64: '@'
65: 'A'
66: 'B'
67: 'C'
68: 'D'
69: 'E'
70: 'F'
71: 'G'
72: 'H'
73: 'I'
74: 'J'
75: 'K'
76: 'L'
77: 'M'
78: 'N'
79: 'O'
80: 'P'
81: 'Q'
82: 'R'
83: 'S'
84: 'T'
85: 'U'
86: 'V'
87: 'W'
88: 'X'
89: 'Y'
90: 'Z'
91: '['
92: '\'
93: ']'
94: '^'
95: '_'
96: '`'
97: 'a'
98: 'b'
99: 'c'
100: 'd'
101: 'e'
102: 'f'
103: 'g'
104: 'h'
105: 'i'
106: 'j'
107: 'k'
108: 'l'
109: 'm'
110: 'n'
111: 'o'
112: 'p'
113: 'q'
114: 'r'
115: 's'
116: 't'
117: 'u'
118: 'v'
119: 'w'
120: 'x'
121: 'y'
122: 'z'
123: '{